In [2]:
from __future__ import print_function
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials

In [22]:
from pathlib import Path
default_base = os.path.split(Path().absolute())[0]
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']
def get_service():
    creds = None
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            path = os.path.join(default_base, 'credentials.json')
            flow = InstalledAppFlow.from_client_secrets_file(
                path, SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())
    return build('gmail', 'v1', credentials=creds)
            

In [51]:
import email
import base64
MAX_RESULT = 511

def search_all_messages(service, user_id):
    try:
        base_functions = service.users().messages().list
        search_id = base_functions(userId=user_id, maxResults=MAX_RESULT).execute()
        pageToken = None
        if 'nextPageToken' in search_id:
            pageToken = search_id['nextPageToken']

        message_list = search_id['messages']
        while pageToken:
            results = base_functions(userId=user_id, maxResults=MAX_RESULT, pageToken=pageToken).execute()
            message_list.append(results['messages'])
            if 'nextPageToken' in results:
                pageToken = results['nextPageToken']
            else:
                break
        print("No more messages")
    except (errors.HttpError, error):
        print("An error occured: %s") % error
    return message_list

## add functionationality to strip the ids 
## add class for ids and threads 

In [ ]:
def get_message_info(service, user_id, msg_id):
    try:
        message = service.users().messages().get(userId=user_id, id=msg_id,format='raw').execute()
        text = base64.urlsafe_b64decode(hello['raw'].encode('ASCII'))
        mime_msg = email.message_from_bytes(text)
        content = mime_msg.get_content_maintype()
        if content == 'multipart':
            text_format, _ = mime_msg.get_payload()
            message_draft = text_format.get_payload()
        elif content == 'text':
            message_draft = mime_msg.get_payload()
        message = message_draft if len(message_draft) >= 500 else "MESSAGE TOO LARGE"
        label_ids = mime_msg['labelIds']
        read_status = "UNREAD" if 'UNREAD' in label_ids else "READ"
        sender = mime_msg["From"]
        subject = mime_msg["subject"]
        received = mime_msg['Received'] 
        return {
                "message_id":msg_id,
                "sender":sender,
                "subject":subject,
                "received":received,
                "message":message,
                "read_status":read_status           
        }
    except Exception:
         print("something") 
        
        
## way of cleaning the messages and received date
## clean sender --> to include email and name
## subject clean the text field
## get dataframe then undertake analysis

In [25]:
service = get_service()

In [45]:
test = service.users().messages().list 

In [47]:
test(userId="me",maxResults=511).execute()

{'messages': [{'id': '179946eb048be76e', 'threadId': '179946eb048be76e'},
  {'id': '179946e6c049f296', 'threadId': '179946e6c049f296'},
  {'id': '17993da0ed2bfa04', 'threadId': '17993da0ed2bfa04'},
  {'id': '17993c4f6b39d91a', 'threadId': '17993c4f6b39d91a'},
  {'id': '179937028d2d8e44', 'threadId': '179937028d2d8e44'},
  {'id': '179936a574723667', 'threadId': '179936a574723667'},
  {'id': '1799367813333dae', 'threadId': '1799367813333dae'},
  {'id': '179933d6a676a54c', 'threadId': '179933d6a676a54c'},
  {'id': '1799304b01a5dfdf', 'threadId': '1799304b01a5dfdf'},
  {'id': '17992bb195491021', 'threadId': '17992bb195491021'},
  {'id': '17990783cdf94ac3', 'threadId': '17990783cdf94ac3'},
  {'id': '179905a4f056eead', 'threadId': '179905a4f056eead'},
  {'id': '1799043cedc3f1f4', 'threadId': '1799043cedc3f1f4'},
  {'id': '1798ffce4309be29', 'threadId': '1798ffce4309be29'},
  {'id': '1798f6035d6479fa', 'threadId': '1798f6035d6479fa'},
  {'id': '1798f49df2cf3655', 'threadId': '1798f49df2cf3655

In [53]:
var = search_all_messages(service,"me")

No more messages


In [54]:
print(type(var))

<class 'list'>


In [90]:
hello = service.users().messages().get(userId='me',id='1798f46655d85c54', format='raw').execute()

In [91]:
hello

{'id': '1798f46655d85c54',
 'threadId': '1798f46655d85c54',
 'labelIds': ['CATEGORY_PROMOTIONS', 'UNREAD', 'INBOX'],
 'snippet': 'Can&#39;t see this email? View online LIVE AT WORTHY FARM 1 DAY TO GO GLOBAL LIVESTREAM - THIS WEEKEND Glastonbury Festival There is just 1 DAY TO GO until we host our exclusive global livestream',
 'sizeEstimate': 19920,
 'raw': 'RGVsaXZlcmVkLVRvOiBidXJyb3VnaGVzZWR3YXJkQGdtYWlsLmNvbQ0KUmVjZWl2ZWQ6IGJ5IDIwMDI6YTE3OjkwNjo0NDA2OjA6MDowOjAgd2l0aCBTTVRQIGlkIHg2Y3NwMTIyNDYxNWVqbzsNCiAgICAgICAgRnJpLCAyMSBNYXkgMjAyMSAwNzoxMzo0NSAtMDcwMCAoUERUKQ0KWC1Hb29nbGUtU210cC1Tb3VyY2U6IEFCZGhQSnp5Y0lNcXRTOU84NTFHd3pkSmxDc2JJaWdsMGowblJYTXhrTitxdUF6eHZ4VVVUS3NrTktLbjlCZkxmUWUycnV3eUV6cTMNClgtUmVjZWl2ZWQ6IGJ5IDIwMDI6YTFjOmYyMTI6OiB3aXRoIFNNVFAgaWQgczE4bXI5MDY2OTIxd21jLjE3Mi4xNjIxNjA2NDI1NjMxOw0KICAgICAgICBGcmksIDIxIE1heSAyMDIxIDA3OjEzOjQ1IC0wNzAwIChQRFQpDQpBUkMtU2VhbDogaT0xOyBhPXJzYS1zaGEyNTY7IHQ9MTYyMTYwNjQyNTsgY3Y9bm9uZTsNCiAgICAgICAgZD1nb29nbGUuY29tOyBzPWFyYy0yMDE2MDgxNjsNCiA

In [67]:
message = base64.urlsafe_b64decode(hello['raw'].encode('ASCII'))

In [66]:
mime_msg = email.message_from_string(message)

TypeError: initial_value must be str or None, not bytes

In [68]:
type(message)

bytes

In [75]:
mime_msg.get_payload()

'multipart'

'LinkedIn | Your job alert for data analyst in London Area, United Kingdom\r\n\r\n19 new jobs match your preferences.\r\n\r\n---------------------------------------------------------\r\n\r\nData Quality Analyst\r\nKDR Recruitment Ltd\r\nLondon, England, United Kingdom\r\n\r\nView job: https://www.linkedin.com/comm/jobs/view/2560923684?alertAction=3D=\r\nmarkasviewed&savedSearchAuthToken=3D1%26AQEhVRHn6cNnqQAAAXmUbmP-ylQk-3cmR8Q=\r\n8lTN_Nzj3kWRKyOp21NwfWOSO5x6-aM_K9OwnDDzzf_5dbsPp2Kq_Nhhuspxh38iEfEdnYvXzbQ=\r\nhsuvBjPHZ8y0ohb7pgkERc0g79bpShzp_-C5cLo0sGrpDKVoXxBLpWm9dA6vSwk1wbdFQ86DvuQ=\r\n2SaBDgNlORPIzmRrjtAr5g7XM7uZ6gn2a-uG700yPZZQkjrjHjlC0Ye_a-mRKHndRJSKfYbouSO=\r\nXsgdo3pSq6ZO8x_Xag03ClQEDWTvjVRky8wHUJLOW2cldCtVm_Q-%26AdiQPZy-K-QO5m5P1Dwm=\r\nsALMHbD2&savedSearchId=3D1269048153&refId=3D423995f6-ff8b-44b0-8c6e-b5b1c44=\r\n3f37e&trackingId=3DHvJnGO%2FpOJG9xaD1WQLNFg%3D%3D&midToken=3DAQHHPav9V-4acw=\r\n&midSig=3D2AYSE_V2TTqFM1&trk=3Deml-email_job_alert_digest_01-job_alert-6-me=\r\nmber

In [84]:
print(mime_msg["From"])
print(mime_msg["subject"])

LinkedIn Job Alerts <jobalerts-noreply@linkedin.com>
19 new jobs for 'data analyst'


In [101]:
import datetime
from dateutil.parser import parse
text = mime_msg['Received']
t = text.split("        ")[1]
time = parse(t)

In [103]:
time.strftime('%Y-%m-%d:%H-%M')

'2021-05-22:07-15'